In [1]:
import numpy as np
import pandas as pd
from urllib.parse import urlparse

import requests
from bs4 import BeautifulSoup
from IPython.display import HTML
# Setting up 'requests' to make HTTPS requests properly takes some extra steps... we'll skip them for now.
requests.packages.urllib3.disable_warnings()
# Import all modules that are required
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import time

In [2]:
def generate_week_strings(start_year, start_week, end_year, end_week):
    week_strings = []

    for year in range(start_year, end_year + 1):
        start = start_week if year == start_year else 1
        end = end_week if year == end_year else 53

        for week in range(start, end + 1):
          #TODO CHANGE LATER range(1, 11)
          for num in range(1, 2):
            #TODO CHANGE LATER range(1, 11)
            week_string = f"week/{year}/{week}/{num}"
            week_strings.append(week_string)

    return week_strings

start_year = 2023
start_week = 20
end_year = 2023
end_week = 20

week_strings = generate_week_strings(start_year, start_week, end_year, end_week)
# for week_string in week_strings:
#     print(week_string)

In [3]:
len(week_strings)

1

In [4]:
week_strings

['week/2023/20/1']

In [5]:
list_of_articles = []
list_of_time_mk_urls = []
for week_string in week_strings:
  week_url = "https://time.mk/" + week_string
  response = requests.get(week_url)
  response.encoding = 'utf-8'
  soup = BeautifulSoup(response.text, "html.parser")
  list_of_articles.extend(soup.find_all(class_="cluster"))
for article in list_of_articles:
  first_anchor_tag = article.select_one('a')
  list_of_time_mk_urls.append("https://time.mk/" + first_anchor_tag.get("href"))

In [6]:
print(list_of_time_mk_urls)

['https://time.mk/r/5a77a87be7/d7c766f4f9/', 'https://time.mk/r/0adcec4ef4/e4b44dbab5/', 'https://time.mk/r/de484df152/aee4b024e0/', 'https://time.mk/r/05cad48bea/f4b6630e96/', 'https://time.mk/r/07b7f45ac8/9de97a7703/', 'https://time.mk/r/96c26694c5/1726ae7360/', 'https://time.mk/r/cdbba32384/db2d54c17e/', 'https://time.mk/r/e98644b822/365c656596/', 'https://time.mk/r/54e04bb94a/fd26db68ca/', 'https://time.mk/r/94e3561430/025647dcd2/']


In [7]:
len(list_of_time_mk_urls)

10

In [8]:

def parseVecerNews(url, content):
    
  soup = BeautifulSoup(content, 'html.parser')

  title = soup.find(class_='entry-title').text.strip() if soup.find(class_='entry-title') else None
  # subtitle = soup.find(class_='td-post-sub-title').text.strip() if soup.find(class_='td-post-sub-title') else None
  subtitle = soup.find(class_='entry-title').text.strip() if soup.find(class_='entry-title') else None
  date = soup.find(class_='entry-date updated td-module-date').text.strip() if soup.find(class_='entry-date updated td-module-date') else None
  main_content = soup.find(class_='td-post-content').text.strip() if soup.find(class_='td-post-content') else None

  news_data = {
      "url" : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseNovaMakedonijaNews(content):
  
  soup = BeautifulSoup(url, content, 'html.parser')


  title = soup.find("h1", class_='entry-title').text.strip() if soup.find("h1", class_='entry-title') else None
  subtitle = soup.find("h1", class_='entry-title').text.strip() if soup.find("h1", class_='entry-title') else None
  date = soup.find(class_='entry-date updated td-module-date').text.strip() if soup.find(class_='entry-date updated td-module-date') else None

  news_paragraphs = soup.find_all('p')
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

  news_data = {
      'url' : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseTv21News(url, content):

  soup = BeautifulSoup(content, 'html.parser')
  title = soup.find(class_='tdb-title-text').text.strip() if soup.find(class_='tdb-title-text') else None
  subtitle = soup.find(class_='tdb-title-text').text.strip() if soup.find(class_='tdb-title-text') else None
  date = soup.find(class_='entry-date updated td-module-date').text.strip() if soup.find(class_='entry-date updated td-module-date') else None

  news_paragraphs = soup.find_all('p')
  # Accumulate the text from all news paragraphs
  main_content = '\n'.join(paragraph.get_text() for paragraph in news_paragraphs)

  news_data = {
      "url" : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parse360stepeni(url, content):

  soup = BeautifulSoup(content, 'html.parser')
  title = soup.find(class_='txt--white txt--center').text.strip() if soup.find(class_='txt--white txt--center') else None
  subtitle = soup.find(class_='txt--white txt--center').text.strip() if soup.find(class_='txt--white txt--center') else None
  date = soup.find(class_='single__post-meta single__post_author txt--center').text.strip() if soup.find(class_='single__post-meta single__post_author txt--center') else None
  main_content = soup.find(class_='entry-content').text.strip() if soup.find(class_='entry-content') else None
  news_data = {
      "url" : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseAlfaNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')

  title = soup.find(class_='entry-title').text.strip() if soup.find(class_='entry-title') else None
  subtitle = soup.find(class_='entry-title').text.strip() if soup.find(class_='entry-title') else None
  date = soup.find(class_='item-metadata posts-date').text.strip() if soup.find(class_='item-metadata posts-date') else None
  main_content = soup.find(class_='entry-content').text.strip() if soup.find(class_='entry-content') else None

  news_data = {
      "url" : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseSitelNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')


  title = soup.find('div', class_='panel-pane pane-node-title').find("h1").text.strip() if soup.find('div', class_='panel-pane pane-node-title').find("h1") else None
  subtitle = soup.find('div', class_='panel-pane pane-node-title').find("h1").text.strip() if soup.find('div', class_='panel-pane pane-node-title').find("h1") else None
  date = soup.find(class_='data-na-objava').text.strip() if soup.find(class_='data-na-objava') else None

  news_paragraphs = soup.find_all('p')
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

  news_data = {
      "url" : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseSdkNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')


  title = soup.find(class_='post-title entry-title').text.strip() if soup.find(class_='post-title entry-title') else None
  subtitle = soup.find(class_='post-title entry-title').text.strip() if soup.find(class_='post-title entry-title') else None
  date = soup.find("span", class_='value').text.strip() if soup.find("span", class_='value') else None

  news_paragraphs = soup.find_all('p')
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

  news_data = {
      "url" : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseFokusNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')


  title = soup.find(class_='post-title single-post-title entry-title').text.strip() if soup.find(class_='post-title single-post-title entry-title') else None
  subtitle = soup.find(class_='post-title single-post-title entry-title').text.strip() if soup.find(class_='post-title single-post-title entry-title') else None
  date = soup.find(class_='entry-date published').text.strip() if soup.find(class_='entry-date published') else None

  news_paragraphs = soup.find(class_="inner-post-entry entry-content").find_all('p')
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

  news_data = {
      "url" : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseSport1News(url, content):

  soup = BeautifulSoup(content, 'html.parser')


  title = soup.find(class_='title').text.strip() if soup.find(class_='title') else None
  subtitle = soup.find(class_='teaser field').text.strip() if soup.find(class_='teaser field') else None
  date = soup.find(class_='submitted').text.strip() if soup.find(class_='submitted') else None

  news_paragraphs = soup.find_all('p')
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

  news_data = {
      "url" : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseKanal5News(url, content):

  #Need to decode first from bytes to text
  html_content = content.decode("utf-8")
  soup = BeautifulSoup(html_content, 'html.parser')


  title = soup.find("h1", class_='article-title').text.strip() if soup.find("h1", class_='article-title') else None
  subtitle = soup.find(class_='subtitle').text.strip() if soup.find(class_='subtitle') else None
  date = soup.find(class_='article-published').text.strip() if soup.find(class_='article-published') else None
  main_content = soup.find(class_ = 'article-text').text.strip() if soup.find(class_ = 'article-text') else None

  news_data = {
      "url" : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseRepublikaNews(url, content):

  #Need to decode first from bytes to text
  html_content = content.decode("utf-8")
  soup = BeautifulSoup(html_content, 'html.parser')


  title = soup.find("h1", class_='title mb-3').text.strip() if soup.find("h1", class_='title mb-3') else None
  subtitle = soup.find("h1", class_='title mb-3').text.strip() if soup.find("h1", class_='title mb-3') else None
  date = soup.find("span", class_='date').text.strip() if soup.find("span", class_='date') else None

  news_paragraphs = soup.find_all('p')
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

  news_data = {
      "url" : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parsePuls24News(url, content):

  #Need to decode first from bytes to text
  html_content = content.decode("utf-8")
  soup = BeautifulSoup(html_content, 'html.parser')


  title = soup.find("h1", class_='entry-title').text.strip() if soup.find("h1", class_='entry-title') else None
  subtitle = soup.find("h1", class_='entry-title').text.strip() if soup.find("h1", class_='entry-title') else None
  date = soup.find(class_='entry-date updated published').text.strip() if soup.find(class_='entry-date updated published') else None

  news_paragraphs = soup.find_all('p')
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

  news_data = {
      "url" : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseMkdNews(url, content):

  #Need to decode first from bytes to text
  html_content = content.decode("utf-8")
  soup = BeautifulSoup(html_content, 'html.parser')


  title = soup.find("h1", class_='main-article-title').text.strip() if soup.find("h1", class_='main-article-title') else None
  subtitle = soup.find("div", id='podnaslov').text.strip() if soup.find("div", id='podnaslov') else None
  date = soup.find(class_='row mx-0 article-info mb-3').find("div").find("div").text.strip() if soup.find(class_='row mx-0 article-info mb-3').find("div").find("div") else None

  news_paragraphs = soup.find_all('p')
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

  news_data = {
      "url" : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseAlsatNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')


  title = soup.find("h1", class_='post-title entry-title').text.strip() if soup.find("h1", class_='post-title entry-title') else None
  subtitle = soup.find(class_='entry-sub-title').text.strip() if soup.find(class_='entry-sub-title') else None
  date = soup.find(class_='date meta-item tie-icon').text.strip() if soup.find(class_='date meta-item tie-icon') else None

  news_paragraphs = soup.find_all('p')
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

  news_data = {
      'url' : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseSlobodnaEvropaNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')


  title = soup.find("h1", class_='title pg-title').text.strip() if soup.find("h1", class_='title pg-title') else None
  subtitle = soup.find("h1", class_='title pg-title').text.strip() if soup.find("h1", class_='title pg-title') else None
  date = soup.find("time").text.strip() if soup.find("time") else None
  main_content = soup.find(class_="wsw").text.strip() if soup.find(class_="wsw") else None

  news_data = {
      'url' : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseMakfaksNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')


  title = soup.find("h1", class_='mvp-post-title left entry-title').text.strip() if soup.find("h1", class_='mvp-post-title left entry-title') else None
  subtitle = soup.find("h1", class_='mvp-post-title left entry-title').text.strip() if soup.find("h1", class_='mvp-post-title left entry-title') else None
  date = soup.find(class_="post-date updated").text.strip() if soup.find(class_="post-date updated") else None

  news_paragraphs = soup.find(id="mvp-content-main").find_all("p")
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

  news_data = {
      'url' : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseMkVoaNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')


  title = soup.find("h1", class_='title pg-title').text.strip() if soup.find("h1", class_='title pg-title') else None
  subtitle = soup.find("h1", class_='title pg-title').text.strip() if soup.find("h1", class_='title pg-title') else None
  date = soup.find("time").text.strip() if soup.find("time") else None

  news_paragraphs = soup.find(class_="wsw").find_all("p")
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

  news_data = {
      'url' : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseNetpressNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')


  title = soup.find("h1", class_='entry-title').text.strip() if soup.find("h1", class_='entry-title') else None
  subtitle = soup.find("h1", class_='entry-title').text.strip() if soup.find("h1", class_='entry-title') else None
  date = soup.find(class_="entry-date updated td-module-date").text.strip() if soup.find(class_="entry-date updated td-module-date") else None

  news_paragraphs = soup.find(class_="td-post-content").find_all("p")
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

  news_data = {
      'url' : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseSlobodenPecatNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')


  title = soup.find("h1", class_='post-main-title').text.strip() if soup.find("h1", class_='post-main-title') else None
  subtitle = soup.find("h1", class_='post-main-title').text.strip() if soup.find("h1", class_='post-main-title') else None
  date = soup.find("time")["datetime"].strip() if soup.find("time") else None

  news_paragraphs = soup.find(class_="post-body-inner").find_all("p")
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

  news_data = {
      'url' : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseMakpressNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')

  title = soup.find("div", class_='page-header').find("h1").text.strip() if soup.find("div", class_='page-header') else None
  subtitle = soup.find("div", class_='page-header').find("h1").text.strip() if soup.find("div", class_='page-header').find("h1") else None
  date = soup.find("div", class_="time").text.strip() if soup.find("div", class_="time") else None

  news_paragraphs = soup.find_all("p")
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs[:-1])

  news_data = {
      'url' : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseDWNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')

  title = soup.select_one(".content-area").find("header").find("h1").text.strip() if soup.select_one(".content-area").find("header").find("h1") else None
  subtitle = soup.select_one(".content-area").find("header").find("h1").text.strip() if soup.select_one(".content-area").find("header").find("h1") else None
  date = soup.find("time").text.strip() if soup.find("time") else None

  news_paragraphs = soup.select_one(".rich-text.has-italic").find_all("p")
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs[:-1])

  news_data = {
      'url' : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parse24MkNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')

  title = soup.find(class_="post-title-area").find("h2", class_='post-title').text.strip() if soup.find(class_="post-title-area").find("h2", class_='post-title') else None
  subtitle = soup.find(class_="post-title-area").find("h2", class_='post-title').text.strip() if soup.find(class_="post-title-area").find("h2", class_='post-title') else None
  date = soup.find("span", class_="post-date").text.strip() if soup.find("span", class_="post-date") else None

  news_paragraphs = soup.find(class_="entry-content").find_all("p")
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs[:-1])

  news_data = {
      'url' : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseRadioMofNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')


  title = soup.find("h1", class_='post-title').text.strip() if soup.find("h1", class_='post-title') else None
  subtitle = soup.find("h1", class_='post-title').text.strip() if soup.find("h1", class_='post-title') else None
  date = soup.find("time")["datetime"].strip() if soup.find("time")["datetime"] else None

  news_paragraphs = soup.find(class_="post-content description").find_all("p")
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs[:-1])

  news_data = {
      'url' : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseCentarMkNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')


  title = soup.find("h1", class_='entry-title penci-entry-title penci-title-center').text.strip() if soup.find("h1", class_='entry-title penci-entry-title penci-title-center') else None
  subtitle = soup.find("h1", class_='entry-title penci-entry-title penci-title-center').text.strip() if soup.find("h1", class_='entry-title penci-entry-title penci-title-center') else None
  date = soup.find("time")["datetime"].strip() if soup.find("time")["datetime"] else None

  news_paragraphs = soup.find(class_="penci-entry-content entry-content").find_all("p")
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs[:-1])

  news_data = {
      'url' : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseA1OnNews(url, content):

  soup = BeautifulSoup(content, 'html.parser')


  title = soup.find("h1", class_='entry-title penci-entry-title').text.strip() if soup.find("h1", class_='entry-title penci-entry-title') else None
  subtitle = soup.find("h1", class_='entry-title penci-entry-title').text.strip() if soup.find("h1", class_='entry-title penci-entry-title') else None
  date = soup.find("time")["datetime"].strip() if soup.find("time")["datetime"] else None

  news_paragraphs = soup.find(class_="penci-entry-content entry-content").find_all("p")
  # Accumulate the text from all news paragraphs
  main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs[:-1])

  news_data = {
      'url' : url,
      'title': title,
      'subtitle': subtitle,
      'date': date,
      'main_content': main_content
  }

  return news_data

def parseTelmaNews(url, content):

    soup = BeautifulSoup(content, 'html.parser')
    title = soup.find(class_="entry-title penci-entry-title penci-title-").text.strip() if soup.find(class_="entry-title penci-entry-title penci-title-") else None
    subtitle = soup.find(class_="entry-title penci-entry-title penci-title-").text.strip() if soup.find(class_="entry-title penci-entry-title penci-title-") else None
    date = soup.find("time")["datetime"].strip() if soup.find("time")["datetime"] else None

    news_paragraphs = soup.find(class_="penci-entry-content entry-content").find_all("p")
    # Accumulate the text from all news paragraphs
    main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

    news_data = {
        'url' : url,
        'title': title,
        'subtitle': subtitle,
        'date': date,
        'main_content': main_content
    }
    
    return news_data

def parseEMagazinNews(url, content):

    soup = BeautifulSoup(content, 'html.parser')
    title = soup.find(class_="entry-title penci-entry-title penci-title-").text.strip() if soup.find(class_="entry-title penci-entry-title penci-title-") else None
    subtitle = soup.find(class_="entry-title penci-entry-title penci-title-").text.strip() if soup.find(class_="entry-title penci-entry-title penci-title-") else None
    date = soup.find("time")["datetime"].strip() if soup.find("time")["datetime"] else None

    news_paragraphs = soup.find(class_="penci-entry-content entry-content").find_all("p")
    # Accumulate the text from all news paragraphs
    main_content = ' '.join(paragraph.get_text() for paragraph in news_paragraphs)

    news_data = {
        'url' : url,
        'title': title,
        'subtitle': subtitle,
        'date': date,
        'main_content': main_content
    }
    
    return news_data

In [9]:
from requests.exceptions import ConnectTimeout, ConnectionError
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
session = requests.Session()
retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
adapter = HTTPAdapter(max_retries=retries)
session.mount('http://', adapter)
session.mount('https://', adapter)

successfully_accessed_sites = []
successfully_accessed_hostnames = []
unsuccessfully_accessed_sites = []

real_urls_list = []
parsedTexts = []


for time_url in list_of_time_mk_urls:
    try:
          response = requests.get(time_url)
          # Gets the real url for each time.mk url.
          real_url = response.url
          real_urls_list.append(real_url)
          if response.status_code!=200:
                unsuccessfully_accessed_sites.append(real_url)
          else:
            successfully_accessed_sites.append(real_url)
            successfully_accessed_hostnames.append(urlparse(real_url).netloc)
            # Gets the html content for each response
            content = response.content
            # Gets the hostname for each url
            hostname = urlparse(real_url).netloc
            if hostname == "www.vecer.press":
                parsedTexts.append(parseVecerNews(real_url, content))
            elif hostname == "mk.tv21.tv":
                parsedTexts.append(parseTv21News(real_url, content))
            elif hostname == "360stepeni.mk":
                parsedTexts.append(parse360stepeni(real_url, content))
            elif hostname == "alfa.mk":
                parsedTexts.append(parseAlfaNews(real_url, content))
            elif hostname == "sitel.com.mk":
                parsedTexts.append(parseSitelNews(real_url, content))
            elif hostname == "sdk.mk":
                parsedTexts.append(parseSdkNews(real_url, content))
            elif hostname == "fokus.mk":
                parsedTexts.append(parseFokusNews(real_url, content))
            elif hostname == "sport1.mk":
                parsedTexts.append(parseSport1News(real_url, content))
            elif hostname == "kanal5.com.mk":
                parsedTexts.append(parseKanal5News(real_url, content))
            elif hostname == "republika.mk":
                parsedTexts.append(parseRepublikaNews(real_url, content))
            elif hostname == "puls24.mk":
                parsedTexts.append(parsePuls24News(real_url, content))
            elif hostname == "mkd.mk":
                parsedTexts.append(parseMkdNews(real_url, content))
            elif hostname == "novamakedonija.com.mk":
                parsedTexts.append(parseNovaMakedonijaNews(real_url, content))
            elif hostname == "alsat.mk":
                parsedTexts.append(parseAlsatNews(real_url, content))
            elif hostname == "www.slobodnaevropa.mk":
                parsedTexts.append(parseSlobodnaEvropaNews(real_url, content))
            elif hostname == "makfax.com.mk":
                parsedTexts.append(parseMakfaksNews(real_url, content))
            elif hostname == "mk.voanews.com":
                parsedTexts.append(parseMkVoaNews(real_url, content))
            elif hostname == "netpress.com.mk":
                parsedTexts.append(parseNetpressNews(real_url, content))
            elif hostname == "www.slobodenpecat.mk":
                parsedTexts.append(parseSlobodenPecatNews(real_url, content))
            elif hostname == "makpress.mk":
                parsedTexts.append(parseMakpressNews(real_url, content))
            elif hostname == "www.dw.com":
                parsedTexts.append(parseDWNews(real_url, content))
            elif hostname == "24.mk":
                parsedTexts.append(parse24MkNews(real_url, content))
            elif hostname == "www.radiomof.mk":
                parsedTexts.append(parseRadioMofNews(real_url, content))
            elif hostname == "centar.mk":
                parsedTexts.append(parseCentarMkNews(real_url, content))
            elif hostname == "a1on.mk":
                parsedTexts.append(parseA1OnNews(real_url, content))
            elif hostname == "telma.com.mk":
                parsedTexts.append(parseTelmaNews(real_url, content))
            elif hostname == "emagazin.mk":
                parsedTexts.append(parseEMagazinNews(real_url, content))
            else:
                parsedTexts.append("Parse function for hostname " + hostname + " doesn't exist yet.")
    except (ConnectTimeout, ConnectionError) as e:
        print(f"Error connecting to {time_url}: {str(e)}")
        unsuccessfully_accessed_sites.append(time_url)
        continue


In [10]:
len(real_urls_list)

10

In [11]:
print(successfully_accessed_sites)
print(successfully_accessed_hostnames)
print(unsuccessfully_accessed_sites)

['https://www.vecer.press/%D1%81%D0%B0%D0%BD%D0%B8%D1%80%D0%B0%D1%9A%D0%B5%D1%82%D0%BE-%D0%BD%D0%B0-%D0%BC%D0%BE%D1%81%D1%82%D0%BE%D1%82-%D0%B1%D0%B5%D0%BB%D0%B0%D1%81%D0%B8%D1%86%D0%B0-%D0%BD%D0%B0%D1%98/', 'https://telma.com.mk/2023/05/21/sonchevo-so-mala-do-umerena-oblachnost-popladne-nestabilno-so-lokalen-poroen-dozhd-grmotrvici-i-zasilen-veter-vremenska-prognoza-za-22-5-2023/', 'https://mk.tv21.tv/ssm-ke-gi-ospori-zakonite-pred-ustaven-sud-peshevski-vladata-ke-gi-vraka-rabotnitsite-vo-robovladetelstvo/', 'https://360stepeni.mk/bajden-chlenkite-na-g7-dogovorija-zaednichki-pristap-kon-kina/', 'https://telma.com.mk/2023/05/21/turskata-ambasada-polna-so-glasachi-se-bira-megju-erdogan-i-kilichdaroglu-vo-vtoriot-krug-na-pretsedatelski-izbori/', 'https://alfa.mk/%D0%BD%D0%B5%D1%83%D1%81%D0%BF%D0%B5%D1%88%D0%BD%D0%B8-%D0%BE%D0%B1%D0%B8%D0%B4%D0%B8-%D0%BD%D0%B0-%D0%BA%D0%BE%D0%B2%D0%B0%D1%87%D0%B5%D0%B2%D1%81%D0%BA%D0%B8-%D0%B4%D0%B0-%D0%B4%D0%BE%D0%B1%D0%B8/?utm_source=rss&utm_medium=rss&

In [12]:
len(successfully_accessed_sites)

10

In [13]:
parsedTexts

[{'url': 'https://www.vecer.press/%D1%81%D0%B0%D0%BD%D0%B8%D1%80%D0%B0%D1%9A%D0%B5%D1%82%D0%BE-%D0%BD%D0%B0-%D0%BC%D0%BE%D1%81%D1%82%D0%BE%D1%82-%D0%B1%D0%B5%D0%BB%D0%B0%D1%81%D0%B8%D1%86%D0%B0-%D0%BD%D0%B0%D1%98/',
  'title': 'Санирањето на мостот „Беласица“ најрано може да заврши до 15 јули',
  'subtitle': 'Санирањето на мостот „Беласица“ најрано може да заврши до 15 јули',
  'date': '21/05/2023 - 12:37',
  'main_content': 'На барање на Град Скопје, Институтот за сеизмички отпорни градби и климатски промени на професорот Михаил Гаревски ќе направи експертиза и проверка на сигурноста и оштетувањата на мостот „Беласица“ кај ТЦ „Ист Гејт“. Според Гаревски, доколку проверката покаже дека има само неконструктивни оштетувања, санирањето најоптимистички може да биде завршено до 10 – 15 јули, а ако деталните анализи покажат потреба од посериозна санација, не е возможно да се заврши за помалку од шест месеци\n\xa0\n\xa0\nНа барање на Град Скопје, Институтот за сеизмички отпорни градби и клима

In [14]:
news_items = parsedTexts.copy()

In [15]:
news_items

[{'url': 'https://www.vecer.press/%D1%81%D0%B0%D0%BD%D0%B8%D1%80%D0%B0%D1%9A%D0%B5%D1%82%D0%BE-%D0%BD%D0%B0-%D0%BC%D0%BE%D1%81%D1%82%D0%BE%D1%82-%D0%B1%D0%B5%D0%BB%D0%B0%D1%81%D0%B8%D1%86%D0%B0-%D0%BD%D0%B0%D1%98/',
  'title': 'Санирањето на мостот „Беласица“ најрано може да заврши до 15 јули',
  'subtitle': 'Санирањето на мостот „Беласица“ најрано може да заврши до 15 јули',
  'date': '21/05/2023 - 12:37',
  'main_content': 'На барање на Град Скопје, Институтот за сеизмички отпорни градби и климатски промени на професорот Михаил Гаревски ќе направи експертиза и проверка на сигурноста и оштетувањата на мостот „Беласица“ кај ТЦ „Ист Гејт“. Според Гаревски, доколку проверката покаже дека има само неконструктивни оштетувања, санирањето најоптимистички може да биде завршено до 10 – 15 јули, а ако деталните анализи покажат потреба од посериозна санација, не е возможно да се заврши за помалку од шест месеци\n\xa0\n\xa0\nНа барање на Град Скопје, Институтот за сеизмички отпорни градби и клима

In [16]:
for news_item in news_items:
    news_item['main_content'] = news_item['main_content'].replace('\n', '') if news_item['main_content'] != None else None
    news_item['main_content'] = news_item['main_content'].replace('\xa0', '') if news_item['main_content'] != None else None
    news_item['title'] = news_item['title'].replace('\xa0', '') if news_item['title'] != None else None
    news_item['subtitle'] = news_item['subtitle'].replace('\xa0', '') if news_item['subtitle'] != None else None

In [17]:
url_text_map = {}
for news_item in news_items:
  entry_url = news_item["url"]
  text = f"{news_item['title']}. {news_item['subtitle']}. {news_item['main_content']}"
  url_text_map[entry_url] = text

In [18]:
print(url_text_map)

{'https://www.vecer.press/%D1%81%D0%B0%D0%BD%D0%B8%D1%80%D0%B0%D1%9A%D0%B5%D1%82%D0%BE-%D0%BD%D0%B0-%D0%BC%D0%BE%D1%81%D1%82%D0%BE%D1%82-%D0%B1%D0%B5%D0%BB%D0%B0%D1%81%D0%B8%D1%86%D0%B0-%D0%BD%D0%B0%D1%98/': 'Санирањето на мостот „Беласица“ најрано може да заврши до 15 јули. Санирањето на мостот „Беласица“ најрано може да заврши до 15 јули. На барање на Град Скопје, Институтот за сеизмички отпорни градби и климатски промени на професорот Михаил Гаревски ќе направи експертиза и проверка на сигурноста и оштетувањата на мостот „Беласица“ кај ТЦ „Ист Гејт“. Според Гаревски, доколку проверката покаже дека има само неконструктивни оштетувања, санирањето најоптимистички може да биде завршено до 10 – 15 јули, а ако деталните анализи покажат потреба од посериозна санација, не е возможно да се заврши за помалку од шест месециНа барање на Град Скопје, Институтот за сеизмички отпорни градби и климатски промени на професорот Михаил Гаревски ќе направи експертиза и проверка на сигурноста и оштетувањ

In [19]:
!pip install transformers
!pip install nlp
!pip install sentencepiece
!pip install torch
!pip install spacy
!pip install vaderSentiment
!pip install textblob

In [20]:
# !python -m spacy download en_core_web_sm

In [21]:
import transformers
import nlp
import torch
import spacy
from transformers import pipeline
from transformers import MarianMTModel, MarianTokenizer

import nltk
# nltk.download('punkt')

In [22]:
# Deprecated

# def chunk_text(text, tokenizer, max_length=1024):
#     # Split the text into sentences
#     sentences = nltk.sent_tokenize(text)
    
#     chunks = []
#     current_chunk = []
#     current_length = 0
    
#     for sentence in sentences:
#         # Tokenize the sentence to get its length
#         sentence_length = len(tokenizer.tokenize(sentence))
        
#         if current_length + sentence_length <= max_length:
#             current_chunk.append(sentence)
#             current_length += sentence_length
#         else:
#             chunks.append(" ".join(current_chunk))
#             current_chunk = [sentence]
#             current_length = sentence_length
    
#     if current_chunk:
#         chunks.append(" ".join(current_chunk))
    
#     return chunks


In [23]:
# METHOD 1: Translate using MarianMT, A framework for translation models
# Helsinki-NLP/opus-mt-mk-en

# Load the pre-trained model and tokenizer
model_name = "Helsinki-NLP/opus-mt-mk-en"
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

In [24]:
start_time = time.time()

url_text_list = list(url_text_map.items())
marian_translated_texts = {}
for url_text in url_text_list[:1]:

    # Define the input Macedonian text
    
    input_url = url_text[0]
    input_text = url_text[1]

    # Split text into sentences using the Natural Language Toolkit Library
    sentences = nltk.sent_tokenize(input_text)
    
    # Translate each sentence separately
    translated_sentences = []
    for sentence in sentences:
        input_ids = tokenizer.encode(sentence, return_tensors="pt")
        with torch.no_grad():
            translated_ids = model.generate(input_ids)
        translated_sentence = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
        translated_sentences.append(translated_sentence)

    # Combine translated sentences
    translated_text = " ".join(translated_sentences)
    marian_translated_texts[input_url] = translated_text
    
    
end_time = time.time()
total_time = end_time - start_time

print(f"The total time to translate the text is {total_time} seconds.")

The total time to translate the text is 47.984381437301636 seconds.


In [25]:
marian_translated_texts

{'https://www.vecer.press/%D1%81%D0%B0%D0%BD%D0%B8%D1%80%D0%B0%D1%9A%D0%B5%D1%82%D0%BE-%D0%BD%D0%B0-%D0%BC%D0%BE%D1%81%D1%82%D0%BE%D1%82-%D0%B1%D0%B5%D0%BB%D0%B0%D1%81%D0%B8%D1%86%D0%B0-%D0%BD%D0%B0%D1%98/': "The suning of the Belasica Bridge can be completed by July 15 at the earliest. The suning of the Belasica Bridge can be completed by July 15 at the earliest. At the request of the City of Skopje, the Institute of Seismic resistance buildings and climate change by Professor Mikhail Garevski will make expertise and inspection of the security and damage of the Belgrade Bridge at TC East Gate. According to Garevski, if the screening shows that there is only non-constructive damage, the screening of the most optimistic can be completed by July 10, and if detailed analysis shows that there is only a need for more serious sanancies, it is not possible to finish in less than six months at the request of City Skopje, the Institute for Seismic resistance buildings and climate change of Prof

In [26]:
!pip install googletrans==4.0.0rc1

In [27]:
# METHOD 2: Translate using Google Translate API
from googletrans import Translator

start_time = time.time()
google_translated_texts = {}
translator = Translator()
for url_text in url_text_list:
    translated_text = translator.translate(url_text[1][:5000], src='mk', dest='en').text
    google_translated_texts[url_text[0]] = translated_text

end_time = time.time()
total_time = end_time - start_time

print(f"The total time to translate the text is {total_time} seconds.")

The total time to translate the text is 5.248109817504883 seconds.


In [28]:
print(google_translated_texts)

{'https://www.vecer.press/%D1%81%D0%B0%D0%BD%D0%B8%D1%80%D0%B0%D1%9A%D0%B5%D1%82%D0%BE-%D0%BD%D0%B0-%D0%BC%D0%BE%D1%81%D1%82%D0%BE%D1%82-%D0%B1%D0%B5%D0%BB%D0%B0%D1%81%D0%B8%D1%86%D0%B0-%D0%BD%D0%B0%D1%98/': 'The repair of the Belasica Bridge may end up by July 15.The repair of the Belasica Bridge may end up by July 15.At the request of the City of Skopje, the Institute for Seismic Resistant Buildings and Climate Change of Professor Mikhail Garevski will make expertise and check the reliability and damage to the Belasica Bridge at East Gate TC.According to Garevski, if the check shows that there is only non -constructive damage, repairing the most optimistic may be completed by July 10 - 15, and if detailed analysis shows the need for more serious remediation, it is not possible to complete in less than six months of the City of Skopje request,Institute for Seismic Resistant Buildings and Climate Change of Professor Mikhail Garevski will do expertise and check the safety and damage to 

In [29]:
# METHOD 3: Translate using distilled version of the  Facebook Neural Language Model for Low-Resource Languages (NLLB-200)
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", src_lang="mk_Cyrl")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")


In [30]:
start_time = time.time()


url_text_list = list(url_text_map.items())
nllb_translated_texts = {}
for url_text in url_text_list[:1]:
    
    # Define the input Macedonian text
    
    input_url = url_text[0]
    input_text = url_text[1]
    
    sentences = nltk.sent_tokenize(input_text)

    translated_sentences = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt")
        translated_tokens = model.generate(
        **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"]
        )
        translated_sentence = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
        translated_sentences.append(translated_sentence)
    
    # Combine translated sentences
    translated_text = " ".join(translated_sentences)
    nllb_translated_texts[input_url] = translated_text
    
end_time = time.time()
total_time = end_time - start_time

print(f"The total time to translate the text is {total_time} seconds.")

The total time to translate the text is 116.4826169013977 seconds.


In [31]:
nllb_translated_texts

{'https://www.vecer.press/%D1%81%D0%B0%D0%BD%D0%B8%D1%80%D0%B0%D1%9A%D0%B5%D1%82%D0%BE-%D0%BD%D0%B0-%D0%BC%D0%BE%D1%81%D1%82%D0%BE%D1%82-%D0%B1%D0%B5%D0%BB%D0%B0%D1%81%D0%B8%D1%86%D0%B0-%D0%BD%D0%B0%D1%98/': "The sanitization of the Belasitsa Bridge can be completed by July 15. The sanitization of the Belasitsa Bridge can be completed by July 15. On the request of the City of Skopje, the Institute for Seismic Resistance to Smog and Climate Change under Professor Mikhail Garevski will conduct a safety and damage inspection on the Belasitsa Bridge at the East Gate Centre. According to Garewski, if the inspection shows that there are only non-constructive damage, the most optimistically possible remediation can be completed by 10 July, and if detailed analyses show the need for serious rehabilitation, it cannot be completed in less than six months.In order to visit Skopje City, the Institute for Seismic Resistance to Construction and Climate Change of Professor Mikhail Garewski will condu

In [32]:
print(marian_translated_texts)
print(google_translated_texts)
print(nllb_translated_texts)

{'https://www.vecer.press/%D1%81%D0%B0%D0%BD%D0%B8%D1%80%D0%B0%D1%9A%D0%B5%D1%82%D0%BE-%D0%BD%D0%B0-%D0%BC%D0%BE%D1%81%D1%82%D0%BE%D1%82-%D0%B1%D0%B5%D0%BB%D0%B0%D1%81%D0%B8%D1%86%D0%B0-%D0%BD%D0%B0%D1%98/': "The suning of the Belasica Bridge can be completed by July 15 at the earliest. The suning of the Belasica Bridge can be completed by July 15 at the earliest. At the request of the City of Skopje, the Institute of Seismic resistance buildings and climate change by Professor Mikhail Garevski will make expertise and inspection of the security and damage of the Belgrade Bridge at TC East Gate. According to Garevski, if the screening shows that there is only non-constructive damage, the screening of the most optimistic can be completed by July 10, and if detailed analysis shows that there is only a need for more serious sanancies, it is not possible to finish in less than six months at the request of City Skopje, the Institute for Seismic resistance buildings and climate change of Prof

In [33]:
# CONTINUE USING ONLY GOOGLE TRANSLATED TEXTS FOR BEST ACCURACY

In [34]:
google_translated_texts

{'https://www.vecer.press/%D1%81%D0%B0%D0%BD%D0%B8%D1%80%D0%B0%D1%9A%D0%B5%D1%82%D0%BE-%D0%BD%D0%B0-%D0%BC%D0%BE%D1%81%D1%82%D0%BE%D1%82-%D0%B1%D0%B5%D0%BB%D0%B0%D1%81%D0%B8%D1%86%D0%B0-%D0%BD%D0%B0%D1%98/': 'The repair of the Belasica Bridge may end up by July 15.The repair of the Belasica Bridge may end up by July 15.At the request of the City of Skopje, the Institute for Seismic Resistant Buildings and Climate Change of Professor Mikhail Garevski will make expertise and check the reliability and damage to the Belasica Bridge at East Gate TC.According to Garevski, if the check shows that there is only non -constructive damage, repairing the most optimistic may be completed by July 10 - 15, and if detailed analysis shows the need for more serious remediation, it is not possible to complete in less than six months of the City of Skopje request,Institute for Seismic Resistant Buildings and Climate Change of Professor Mikhail Garevski will do expertise and check the safety and damage to 

In [35]:
# Function to aggregate the sentiment for a single entity, since a single entity will have multiple sentiments
# if the entity is present in multiple sentences.

def aggregate_sentiment(sentiments):
    # Initialize counters
    total_positive = 0
    total_negative = 0
    count_positive = 0
    count_negative = 0
    
    for sentiment in sentiments:
        if sentiment['label'] == 'POSITIVE':
            total_positive += sentiment['score']
            count_positive += 1
        elif sentiment['label'] == 'NEGATIVE':
            total_negative += sentiment['score']
            count_negative += 1
            
    # Calculate averages
    avg_positive = total_positive / count_positive if count_positive != 0 else 0
    avg_negative = total_negative / count_negative if count_negative != 0 else 0
    
#     # Determine the overall sentiment
#     if avg_positive > avg_negative:
#         return {'label': 'POSITIVE', 'score': avg_positive}
#     else:
#         return {'label': 'NEGATIVE', 'score': avg_negative}



    if total_positive > total_negative:
        return {'label': 'POSITIVE', 'score': avg_positive, 'total_positive': total_positive, 'total_negative': total_negative, 'score_positive': avg_positive, 'score_negative': avg_negative, 'count_positive': count_positive, 'count_negative': count_negative}
    else:
        return {'label': 'NEGATIVE', 'score': avg_negative, 'total_positive': total_positive, 'total_negative': total_negative, 'score_positive': avg_positive, 'score_negative': avg_negative, 'count_positive': count_positive, 'count_negative': count_negative}

In [36]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define sentiment analysis pipeline using transformers
# sentiment_pipeline = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def extract_entities_and_sentiment(text):
    doc = nlp(text)
    
    # Store entity sentiments
    entity_sentiments = {}
    # Store sentences for each entity
    entity_sentences = {}

    for ent in doc.ents:
        if ent.text not in entity_sentences:
            entity_sentences[ent.text] = [ent.sent.text]
        else:
            entity_sentences[ent.text].append(ent.sent.text)
            
    for entity, sentences in entity_sentences.items():
        sentiments = []
        for sentence in sentences:
            sentiment = sentiment_pipeline(sentence)
            sentiments.append(sentiment[0])
        
    
        entity_sentiments[entity] = aggregate_sentiment(sentiments)

    return entity_sentiments



In [37]:
entity_sentiments = []
google_translated_texts_list = list(google_translated_texts.items())
url_sentiments = []
for url_translated_text in google_translated_texts_list[:1]:
    entity_sentiments = extract_entities_and_sentiment(url_translated_text[1])
    url_sentiments.append([url_translated_text[0], entity_sentiments])
    

 

In [38]:
url_sentiments

[['https://www.vecer.press/%D1%81%D0%B0%D0%BD%D0%B8%D1%80%D0%B0%D1%9A%D0%B5%D1%82%D0%BE-%D0%BD%D0%B0-%D0%BC%D0%BE%D1%81%D1%82%D0%BE%D1%82-%D0%B1%D0%B5%D0%BB%D0%B0%D1%81%D0%B8%D1%86%D0%B0-%D0%BD%D0%B0%D1%98/',
  {'the Belasica Bridge': {'label': 'NEGATIVE',
    'score': 0.9808381199836731,
    'total_positive': 0.9797104597091675,
    'total_negative': 3.9233524799346924,
    'score_positive': 0.9797104597091675,
    'score_negative': 0.9808381199836731,
    'count_positive': 1,
    'count_negative': 4},
   'July 15.The': {'label': 'NEGATIVE',
    'score': 0.9808381199836731,
    'total_positive': 0,
    'total_negative': 0.9808381199836731,
    'score_positive': 0,
    'score_negative': 0.9808381199836731,
    'count_positive': 0,
    'count_negative': 1},
   'July 15.At': {'label': 'NEGATIVE',
    'score': 0.9808381199836731,
    'total_positive': 0,
    'total_negative': 0.9808381199836731,
    'score_positive': 0,
    'score_negative': 0.9808381199836731,
    'count_positive': 0,
  

In [39]:
# WRITE NEWS TO FILE
################################################################################################################################
# file_path = "final.txt"
# delimiter = "____###DELIMITER###____"  # Choose a delimiter that won't appear in your data

# with open(file_path, "w", encoding="utf-8") as file:
#     # Iterate through the map and write each key-value pair
#     for key, value in translated_texts.items():
#         file.write(f"{key}{delimiter}{value}\n")  # Add a newline character after each pair

# print("Key-value pairs written to the file successfully.")

In [33]:
# READ NEWS FROM FILE
################################################################################################################################
# file_path = "final.txt"
# delimiter = "____###DELIMITER###____"  # The same delimiter you used while saving

# read_map = {}

# with open(file_path, "r", encoding="utf-8") as file:
#     for line in file:
#         key, value = line.strip().split(delimiter, 1)
#         read_map[key] = value

# print("Key-value pairs read from the file:", read_map)

Key-value pairs read from the file: {'https://telma.com.mk/2023/05/14/evopskoto-ne-ne-ja-premisluva-vladata-za-pette-zakoni-so-evropsko-znamence/': 'The European  "no" thinks of the government for the five laws with a European flag. The European  "no" thinks of the government for the five laws with a European flag. Prime Minister Dimitar Kovacevski is still without direct response to the question of whether the government has come forward with the European Commission to put a European flag on the five laws being made to realise the agreement with..Bechtel and Enka, to speed up the construction of sections of Corridors 8 and 10D highways, a move by the government that Brussels has reached clearly from Brussels. I have communication not only with the EU delegation, but also daily communication with the US Embassy, as confirmed yesterday by State Department High Representative Mr. Escobar. We explained that we would no longer allow the cutting of the assembly because there have been only 